In [27]:
%pip install pydot
%brew install graphviz


Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%brew` not found.


In [109]:
from glob import glob
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf

# CNN Building Tools below-these lines are causing problems in the code for some reason
import keras
from keras import layers, models
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras import backend as K

# from images.ipynb import load_images
from keras.backend import clear_session
import plotly.express as px
import plotly.graph_objects as go

In [34]:
df = []
df = pd.read_csv("HAM10000_metadata.csv")
df = df.sort_values(by="image_id")
lesion_type_dict = {
    "nv": "Melanocytic nevi",
    "mel": "Melanoma",
    "bkl": "Benign keratosis-like lesions",
    "akiec": "Actinic keratoses",
    "vasc": "Vascular lesions",
    "df": "Dermatofibroma",
    "bcc": "Basal Cell Carcinoma",
}

In [35]:
# features and target variables for train test split
# features = df[["dx_type", "age", "sex", "localization", "img"]]
features = np.load("images.npy")
target = df[["dx"]]

In [36]:
img = Image.open("HAM10000_images/ISIC_0024306.jpg")
img = np.array(img.resize((65,45)))
print(img[0][0])
print(features[0][0][0])

[229 132 134]
[229 132 134]


In [67]:
target = pd.get_dummies(target, columns=["dx"])

In [68]:
image_x_train = features[:6000]
image_x_val = features[6000:7600]
image_x_test = features[7600:]
image_y_train = target[:6000]
image_y_val = target[6000:7600]
image_y_test = target[7600:]

In [174]:
# Start of CNN Building
input_shape = (45, 65, 3)
num_classes = 7

In [175]:
# Initialising the CNN
model_cnn = Sequential()
# Adding layers to the model
model_cnn.add(layers.InputLayer(input_shape=input_shape))
model_cnn.add(
    Conv2D(
        32,
        kernel_size=(3, 3),
        activation="relu",
        padding="same",
    )
)

model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding="Same"))
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(layers.Dropout(0.25))

model_cnn.add(Conv2D(64, (3, 3), activation="relu", padding="Same"))
model_cnn.add(Conv2D(64, (3, 3), activation="relu", padding="Same"))
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(layers.Dropout(0.25))


model_cnn.add(Conv2D(64, (3, 3), activation="relu", padding="Same"))
model_cnn.add(Conv2D(32, (3, 3), activation="relu", padding="Same"))
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(layers.Dropout(0.25))

model_cnn.add(Flatten())
# model.add(Dense(128, activation="relu"))
model_cnn.add(layers.Dropout(0.25))
model_cnn.add(Dense(128))
model_cnn.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 45, 65, 32)        896       
                                                                 
 conv2d_25 (Conv2D)          (None, 45, 65, 32)        9248      
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 22, 32, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 22, 32, 32)        0         
                                                                 
 conv2d_26 (Conv2D)          (None, 22, 32, 64)        18496     
                                                                 
 conv2d_27 (Conv2D)          (None, 22, 32, 64)        36928     
                                                     

In [177]:
text = df[["age", "localization", "sex"]]
text = pd.get_dummies(text, columns=["localization", "sex"])
text_x_train = np.array(text[:6000])
text_x_test = np.array(text[6000:7600])
text_x_val = np.array(text[7600:])
text_y_train = np.array(target[:6000])
text_y_test = np.array(target[6000:7600])
text_y_val = np.array(target[7600:])
# print(text_x_train[:10])

In [178]:
model_txt = Sequential()
model_txt.add(layers.InputLayer(input_shape=(19,)))
model_txt.add(Dense(200, activation="linear"))
model_txt.add(Dense(128))
model_txt.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 200)               4000      
                                                                 
 dense_36 (Dense)            (None, 128)               25728     
                                                                 
Total params: 29728 (116.12 KB)
Trainable params: 29728 (116.12 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [179]:

# text_side = model_txt(text_x_train)

In [180]:
x_train = np.array([])
# for i in range (0, len(image_x_train)):
#     x_train[i] = text_train[i] + image_x_train[i]
#     x_train = np.array([np.array([text_x_train[i], image_x_train[i]])])

# x_train[0] = np.array([text_x_train[0], image_x_train[0]])

x_train = [[text_x_train[i], image_x_train[i]] for i in range(0, len(image_x_train))]
x_val = [[text_x_val[i], image_x_val[i]] for i in range(0, len(image_x_val))]

In [181]:
tmp = x_train[0][1]
print(len(x_train))
print(type(x_train[0][0]))
print(len(tmp))
print(tmp.shape)
print(type(tmp))
# print(tmp)

6000
<class 'numpy.ndarray'>
45
(45, 65, 3)
<class 'numpy.ndarray'>


In [182]:
img_input = layers.Input(shape=input_shape, dtype=tf.int32, name="img")
txt_input = layers.Input(shape=(19,), dtype=tf.int32, name="txt")
image_side = model_cnn(img_input)
text_side = model_txt(txt_input)

merged = layers.Concatenate()([image_side, text_side])
merged = layers.Dense(256, activation="relu")(merged)
output = layers.Dense(7, activation="softmax", name="class")(merged)
model = models.Model([img_input, txt_input], output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 img (InputLayer)            [(None, 45, 65, 3)]          0         []                            
                                                                                                  
 txt (InputLayer)            [(None, 19)]                 0         []                            
                                                                                                  
 sequential_18 (Sequential)  (None, 128)                  284928    ['img[0][0]']                 
                                                                                                  
 sequential_19 (Sequential)  (None, 128)                  29728     ['txt[0][0]']                 
                                                                                            

In [183]:
# Define the optimizer
# optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0005),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(
    monitor="val_accuracy", patience=3, verbose=1, factor=0.5, min_lr=0.00001
)

In [196]:
epochs = 5
batch_size = 100
history = model.fit(
    [image_x_train, text_x_train],
    text_y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    steps_per_epoch=6000 // batch_size,
    callbacks=[learning_rate_reduction],
)

Epoch 1/5
60/60 [==============================] - 42s 665ms/step - loss: 3.4456 - accuracy: 0.6168 - lr: 5.0000e-04
Epoch 2/5
60/60 [==============================] - 41s 684ms/step - loss: 0.9273 - accuracy: 0.6947 - lr: 5.0000e-04
Epoch 3/5
60/60 [==============================] - 41s 684ms/step - loss: 0.9227 - accuracy: 0.6947 - lr: 5.0000e-04
Epoch 4/5
60/60 [==============================] - 42s 704ms/step - loss: 0.8847 - accuracy: 0.7058 - lr: 5.0000e-04
Epoch 5/5
60/60 [==============================] - 41s 676ms/step - loss: 0.8266 - accuracy: 0.7148 - lr: 5.0000e-04


In [195]:
print(text_y_train.shape)
print(len(x_train[0][0]))

(6000, 7)
19


In [141]:
len(x_train[1][0])

19